In [19]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1").unwrapped

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [21]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [22]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns



In [23]:

def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters())
    optimizerC = optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state, _ = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor, 'actor.pkl')
    torch.save(critic, 'critic.pkl')
    env.close()



In [24]:

if __name__ == '__main__':
    if os.path.exists('actor.pkl'):
        actor = torch.load('actor.pkl')
        print('Actor Model loaded')
    else:
        actor = Actor(state_size, action_size).to(device)
    if os.path.exists('critic.pkl'):
        critic = torch.load('critic.pkl')
        print('Critic Model loaded')
    else:
        critic = Critic(state_size, action_size).to(device)
    trainIters(actor, critic, n_iters=100)

Actor Model loaded
Critic Model loaded
Iteration: 0, Score: 72
Iteration: 1, Score: 28
Iteration: 2, Score: 60
Iteration: 3, Score: 74
Iteration: 4, Score: 40
Iteration: 5, Score: 32
Iteration: 6, Score: 64
Iteration: 7, Score: 44
Iteration: 8, Score: 73
Iteration: 9, Score: 52
Iteration: 10, Score: 42
Iteration: 11, Score: 127
Iteration: 12, Score: 88
Iteration: 13, Score: 51
Iteration: 14, Score: 74
Iteration: 15, Score: 39
Iteration: 16, Score: 42
Iteration: 17, Score: 71
Iteration: 18, Score: 60
Iteration: 19, Score: 85
Iteration: 20, Score: 68
Iteration: 21, Score: 50
Iteration: 22, Score: 67
Iteration: 23, Score: 119
Iteration: 24, Score: 81
Iteration: 25, Score: 64
Iteration: 26, Score: 55
Iteration: 27, Score: 50
Iteration: 28, Score: 89
Iteration: 29, Score: 55
Iteration: 30, Score: 56
Iteration: 31, Score: 86
Iteration: 32, Score: 61
Iteration: 33, Score: 57
Iteration: 34, Score: 92
Iteration: 35, Score: 59
Iteration: 36, Score: 77
Iteration: 37, Score: 35
Iteration: 38, Scor